In [1]:
# change these to try this notebook out
BUCKET = 'gcp_model1'

In [2]:
from keras import models
from keras import layers
import tensorflow as tf   

Using TensorFlow backend.


In [3]:
import tensorflow as tf   
from keras.datasets import boston_housing

#get data
(x_train, y_train), (x_test, y_test) = boston_housing.load_data()

#normalization
mean = x_train.mean(axis=0)
x_train -= mean
std = x_train.std(axis=0)
x_train /= std

x_test -= mean
x_test /= std

def build_model():
    model = tf.keras.Sequential()
    model.add(tf.keras.layers.Dense(64, activation='relu', input_shape=(x_train.shape[1],)))
    model.add(tf.keras.layers.Dense(64, activation='relu'))
    model.add(tf.keras.layers.Dense(1))

    model.compile(optimizer='rmsprop',
              loss='mse')
    return model

model = build_model()
model.summary()

W0815 15:08:28.488514   924 deprecation.py:506] From C:\Users\Jumper.ai\AppData\Local\Continuum\anaconda3\lib\site-packages\tensorflow\python\ops\init_ops.py:1251: calling VarianceScaling.__init__ (from tensorflow.python.ops.init_ops) with dtype is deprecated and will be removed in a future version.
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor


Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense (Dense)                (None, 64)                896       
_________________________________________________________________
dense_1 (Dense)              (None, 64)                4160      
_________________________________________________________________
dense_2 (Dense)              (None, 1)                 65        
Total params: 5,121
Trainable params: 5,121
Non-trainable params: 0
_________________________________________________________________


In [4]:
model.fit(x_train, y_train, epochs=80, batch_size=16, verbose=0)

In [ ]:
print("Test DAta:")
print(x_test[:1])
print("Expected Result:",model.predict(x_test[:1]))

In [ ]:
##saving model to bucket
path="gs://%s/keras-job-dir/keras_export" % BUCKET

# Load the saved keras model back.
tf.keras.experimental.export_saved_model(model, path)